In [1]:
from math import log, sqrt, exp
from scipy import stats
from scipy.optimize import fsolve

In [3]:
class call_option():
    
    def __init__(self, s0, strike, pricing_date, maturity_date, rf, sigma):
        self.s0 = float(s0)
        self.K = strike
        self.t = pricing_date
        self.M = maturity_date
        self.r = rf
        self.sigma = sigma
        
    def update_ttm(self):
        if self.t > self.M:
            raise ValueError("Maturity date should be greater than pricing date.")
        self.T = (self.M - self.t).days / 365
    
    def d1(self):
        ''' Helper function for N(d1) and N(d2) '''
        d1 = ((log(self.S0 / self.K) + (self.r + 0.5 * self.sigma ** 2) * self.T) / (self.sigma * sqrt(self.T)))
        return d1
    
    def value(self):
        ''' Return option value. '''
        self.update_ttm()
        d1 = self.d1()
        d2 = ((log(self.S0 / self.K) + (self.r - 0.5 * self.sigma ** 2) * self.T) / (self.sigma * sqrt(self.T)))
        value = (self.S0 * stats.norm.cdf(d1, 0.0, 1.0) - self.K * exp(-self.r * self.T) * stats.norm.cdf(d2, 0.0, 1.0))
        return value
    
    def vega(self):
        ''' Return Vega of option. '''
        self.update_ttm()
        d1 = self.d1()
        vega = self.S0 * stats.norm.pdf(d1, 0.0, 1.0) * sqrt(self.T)
        return vega
    
    def implied_vol(self, c0):
        """
        Reverse solving the black scholes equation to find out implied vol
        No direct analytical method to solve this, so, we use numerical method to find out solution iteratively  
        """
        sigma_est = 0.2 # first guess for vol
        option = call_option(self.s0, self.K, self.t, self.M, self.r, sigma_est) # create a new call option object using the sigme estimated
        
        option.update_ttm()
        
        def difference(sigma):
            option.sigma = sigma
            return option.value() - c0
        
        iv = fsolve(difference, sigma_est)[0]
        
        return iv       